In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

def get_session(gpu_fraction=0.2):
    
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session())

Using TensorFlow backend.


In [2]:
import scipy.io as sio
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Lambda
from keras.models import Model
import keras


In [3]:
def getConnection_c2v(H):   # using r edges of a variable node to calculate the j edge
    IndexMatrix=np.zeros(H.shape,dtype=int)
    dim=np.sum(H,dtype=int)
    C_c2v=np.zeros((dim,dim),dtype=bool)
    onesPosition=[]
    otherOnesPosition=[]
    onesIndex=-1
     
    for j in range(0,H.shape[1]):
        for i in range(0,H.shape[0]):
            if H[i,j]==1:
                onesIndex+=1
                onesPosition.append(onesIndex)
                
        for k in onesPosition:
            otherOnesPosition=list(onesPosition)
            otherOnesPosition.remove(k)
#             print('onesPosition:',onesPosition)
#             print('otheronesposition:', otherOnesPosition)
            for l in otherOnesPosition:
                C_c2v[l,k]=True
            otherOnesPosition=[]
        onesPosition=[]
#         print('cleared')
#         print(onesPosition)
        
    return C_c2v

def getConnection_v2c(H): ## using j edges of a check node to calculate the r edge
    IndexMatrix=np.zeros(H.shape,dtype=int)
    dim=np.sum(H,dtype=int)
    C_v2c=np.zeros((dim,dim),dtype=bool)
    onesPosition=[]
    otherOnesPosition=[]
    onesIndex=-1
    
    for j in range(0,H.shape[1]):   # set index for ones in H
        for i in range(0,H.shape[0]):
            if H[i,j]==1:
                onesIndex+=1
                IndexMatrix[i,j]=onesIndex
                             
    for i in range(0,H.shape[0]):
        for j in range(0,H.shape[1]):
            if H[i,j]==1:   
               onesPosition.append(IndexMatrix[i,j]) 
            
        for k in onesPosition:
            otherOnesPosition=list(onesPosition)
            otherOnesPosition.remove(k)
#             print('onesPosition:',onesPosition)
#             print('otheronesposition:', otherOnesPosition)
            for l in otherOnesPosition:
                C_v2c[l,k]=True
            otherOnesPosition=[]
        onesPosition=[]
#         print('cleared')
#         print(onesPosition)
    return C_v2c



# there are mistakes!!!!
#
# def getConnection_for_firstLayer(H): #the dimention of C_firstLayer is number of variable nodes x  number of edges
#                                      # from the channel level to calulate the r edge from a check node
#     IndexMatrix=np.zeros(H.shape,dtype=int)
#     dim=np.sum(H,dtype=int)
#     C_firstLayer=np.zeros((H.shape[1],dim),dtype=bool)
#     onesIndex=-1
    
#     for j in range(0,H.shape[1]):   # set index for ones in H
#         for i in range(0,H.shape[0]):
#             if H[i,j]==1:
#                 onesIndex+=1
#                 IndexMatrix[i,j]=onesIndex
                             
#     for i in range(0,H.shape[0]):
#         for j in range(0,H.shape[1]):
#             if H[i,j]==1:
#                 for k in range(0,H.shape[0]):
#                     if k!=i:
#                         if H[k,j]==1:
#                            C_firstLayer[i,IndexMatrix[i,j]]=True
#                            break;
            
#     return C_firstLayer

def getConnection_for_firstLayer(H): #the dimention of C_firstLayer is number of variable nodes x  number of edges
                                     # from the channel level to calulate the r edge from a check node
    IndexMatrix=np.zeros(H.shape,dtype=int)
    dim=np.sum(H,dtype=int)
    C_firstLayer=np.zeros((H.shape[1],dim),dtype=bool)
    onesIndex=-1
    
    for j in range(0,H.shape[1]):   # set index for ones in H
        for i in range(0,H.shape[0]):
            if H[i,j]==1:
                onesIndex+=1
                C_firstLayer[j,onesIndex]=True
                             
            
    return C_firstLayer



def getConnection_for_lastLayer(H): #the dimention of C_lastLayer is number of number of edges x variable nodes
                                     
    IndexMatrix=np.zeros(H.shape,dtype=int)
    dim=np.sum(H,dtype=int)
    C_lastLayer=np.zeros((dim,H.shape[1]),dtype=bool)
    onesIndex=-1
    
    for j in range(0,H.shape[1]):   # set index for ones in H
        for i in range(0,H.shape[0]):
            if H[i,j]==1:
                onesIndex+=1
                C_lastLayer[onesIndex,j]=True
            
    return C_lastLayer


# if a weigt is initialized to zero, 
#in future iterations it will not be updated by gradient descent and backpropagation

# def init_for_v2c_layer(shape,dtype=None):
#     print(shape)
#     #matrix=getConnection_v2c(H)
#     matrix=C_v2c
#     matrix=tf.convert_to_tensor(matrix,dtype=tf.float32)
#     result_matrix=tf.random_normal(shape, dtype=tf.float32)
#     return tf.multiply(result_matrix,matrix)


def init_for_r2q_layer(shape,dtype=None):
#     print('shape: ',shape)
    #matrix=getConnection_c2v(H)  #not need to call the function, a matrix can be globle.
    matrix=C_r2q
    matrix=tf.convert_to_tensor(matrix,dtype=tf.float32)
    #result_matrix=tf.random_normal(shape, dtype=tf.float32)
    result_matrix=tf.ones(shape,dtype=tf.float32)
    return tf.multiply(result_matrix,matrix)


def init_for_firstLayer(shape,dtype=None):
#     print('shape_firstLayer: ',shape)
    matrix=C_firstLayer
    matrix=tf.convert_to_tensor(matrix,dtype=tf.float32)
    #result_matrix=tf.random_normal(shape, dtype=tf.float32)
    result_matrix=tf.ones(shape,dtype=tf.float32)
    return tf.multiply(result_matrix,matrix)


def init_for_lastLayer(shape,dtype=None):
    matrix=C_lastLayer
    matrix=tf.convert_to_tensor(matrix,dtype=tf.float32)
    #result_matrix=tf.random_normal(shape, dtype=tf.float32)
    result_matrix=tf.ones(shape,dtype=tf.float32)
    return tf.multiply(result_matrix,matrix)


def get_r_edges_tensorflow(x,q2r_matrix):
    v2c_matrix=tf.convert_to_tensor(q2r_matrix,dtype=tf.float32)
    v2c_zeros_matrix=tf.zeros(tf.shape(v2c_matrix),dtype=tf.float32)
    v2c_zeros_matrix=tf.equal(v2c_zeros_matrix,tf.transpose(v2c_matrix))
    v2c_zeros_matrix=tf.cast(v2c_zeros_matrix,tf.float32)
    
    x1=tf.abs(x)                              #avoid to be zero
#     x1=tf.log(tf.div((tf.exp(x1)+1),(tf.exp(x1)-1)))
#     x1=tf.matmul(x1,v2c_matrix)                      # x1*v2c_matrix
#     x1=tf.log(tf.div((tf.exp(x1)+1),(tf.exp(x1)-1)))

    x1=-tf.log(tf.tanh(x1/2))
    x1=tf.matmul(x1,v2c_matrix) 
    x1=-tf.log(tf.tanh(x1/2))
    
    x2=tf.sign(x)                             
    x2_zeros=tf.zeros(tf.shape(x),dtype=tf.float32)     #avoid to be zero  make sign(0) to be 1
    x2_zeros=tf.cast(tf.equal(x2_zeros,x2),tf.float32)
    x2=x2+x2_zeros
    
#     print('x2_befor_expand_dims: ',x2)
    x2=tf.expand_dims(x2,1)
    
#     print('x2_after_expand_dims: ',x2)
    
    x2=tf.multiply(x2,tf.transpose(v2c_matrix))+tf.transpose(v2c_zeros_matrix)# should add a matrix with shape(?,n,n)
                                                                         # tf.add supports broadcasting
    x2=tf.transpose(x2,perm=[0, 2, 1])
#     print('v2c_zeros_matrix: ',v2c_zeros_matrix)
#     print('x2: ',x2)
    
    x2=tf.reduce_prod(x2,1) #shape from (?,64) to (1,64) caused error
#     print('x2_prod: ',x2)
    
    return tf.minimum(tf.maximum(tf.multiply(x1,x2),-20),20)



# transform the form of edges-messages matrix
def transform_edges_matrix_to_vector(H,matrix):
    vector=[]
    for j in range(H.shape[1]):
        for i in range(H.shape[0]):
            if H[i,j]==1:
                vector.append(matrix[i,j])
    return np.array(vector,dtype=float)

def addNoise(x, sigma):
    w = sigma*tf.random_normal(tf.shape(x))
    return x + w

# def ber(y_true, y_pred):
#     #return K.mean(K.not_equal(y_true, K.round(y_pred)))\n",
#     return tf.reduce_mean(tf.cast(tf.not_equal(y_true, tf.round(y_pred)),tf.float32))

def BLER_errors(y_true,y_pred):
    c=tf.reduce_sum(tf.cast(tf.not_equal(y_true, tf.round(y_pred)),tf.float32),1)
    c=tf.cast(c,tf.bool)
    c=tf.reduce_sum(tf.cast(c,tf.float32))
    return tf.reduce_sum(c)

In [4]:
#ldpc
# mat_contents = sio.loadmat('mat_files/ldpcH_20x40')
# H = mat_contents['H'].toarray()   #load mat file,and convert it to numpy ndarray
# H=H.astype(int)

#polar codes
mat_contents = sio.loadmat('mat_files/H_Polarcodes_32x64_for_allZeros.mat')
H = mat_contents['H']   #load mat file,and convert it to numpy ndarray
H=H.astype(int)

# H=np.array([[1,1,1,0,1,1],[0,1,0,1,1,0],[0,1,1,1,0,1]],dtype=int)
# mat_contents = sio.loadmat('result')
# result = mat_contents['result']   #load mat file,and convert it to numpy ndarray
# mat_contents = sio.loadmat('H')
# H=mat_contents['H'].toarray()
# H=H.astype(int)
# mat_contents = sio.loadmat('testset')
# x_train=mat_contents['Lc']

n_iteration=6


edges_count=np.sum(H,dtype=int)
input_shape=H.shape[1]
train_sigma=0.6
n_epochs=200
C_firstLayer=getConnection_for_firstLayer(H)
C_q2r=getConnection_v2c(H)
C_r2q=getConnection_c2v(H)
C_r2q=np.concatenate((C_firstLayer, C_r2q), axis=0)
C_lastLayer=getConnection_for_lastLayer(H)
C_lastLayer=np.concatenate((np.eye(input_shape,dtype=bool), C_lastLayer), axis=0)
weights_count=np.sum(C_q2r)
weights_count_outputlayer=np.sum(C_lastLayer)
sigma=0.6

y_train=np.zeros((1280,input_shape),dtype=float)
# y_train=np.array([y_train])
x_train=np.ones((1280,input_shape),dtype=float)
# x_train=x_train+sigma_ch*np.random.randn(40)
# x_train=np.array([x_train])

In [5]:
#build graph

exec('x = tf.placeholder(tf.float32, shape=[None, {}])'.format(input_shape))
exec('y_ = tf.placeholder(tf.float32, shape=[None, {}])'.format(input_shape))

C_firstLayer=tf.constant(C_firstLayer,dtype=tf.float32)
for i in range(n_iteration):
    exec('weights_layer_r2q_{}'.format(i)+'={}')
    
x_noise=addNoise(x,sigma)
q_0=tf.minimum(tf.maximum(tf.matmul(x_noise,C_firstLayer),-20),20)
r_0=get_r_edges_tensorflow(q_0,C_q2r)
r_0=tf.concat([x_noise,r_0],1)

for i in range(1,n_iteration):
    with tf.name_scope('layer_r2q_{}'.format(i)):
        for j in range(weights_count):
            exec('layer_r2q_{}_w{}=tf.Variable(tf.truncated_normal([1,1]),name=\'w{}\')'.format(i,j,j))
        j=0
        for k in range(C_r2q.shape[1]):
            nonzero_indices=np.where(C_r2q[:,k]!=0)[0]
            if nonzero_indices.shape[0] !=0:
                exec('layer{}_q{}=layer_r2q_{}_w{}*tf.expand_dims(r_{}[:,nonzero_indices[0]],1)'
                     .format(i,k,i,j,i-1))
                exec('layer{}_q{}=tf.minimum(tf.maximum(layer{}_q{},-20),20)'.format(i,k,i,k))
                j+=1
                if nonzero_indices.shape[0] !=1:
                    for l in range(1,nonzero_indices.shape[0]):
                        exec('layer{}_q{}=layer{}_q{}+layer_r2q_{}_w{}*tf.expand_dims(r_{}[:,nonzero_indices[{}]],1)'
                             .format(i,k,i,k,i,j,i-1,l))
                        j+=1
                exec('layer{}_q{}=tf.minimum(tf.maximum(layer{}_q{},-20),20)'.format(i,k,i,k))
                
        exec('q_{}=tf.concat([layer{}_q{},layer{}_q{}],1)'.format(i,i,0,i,1))
        for k in range(2,C_r2q.shape[1]):
            exec('q_{}=tf.concat([q_{},layer{}_q{}],1)'.format(i,i,i,k))

    with tf.name_scope('layer_q2r_{}'.format(i)):
        exec('r_{}=tf.concat([x_noise,get_r_edges_tensorflow(q_{},C_q2r)],1)'
             .format(i,i))
                    
with tf.name_scope('output'):
    outputLayer_weights={}
    for j in range(weights_count_outputlayer):
        exec('outputLayer_w{}=tf.Variable(tf.truncated_normal([1,1]),name=\'w{}\')'.format(j,j))
    
    j=0
    for k in range(C_lastLayer.shape[1]):
        nonzero_indices=np.where(C_lastLayer[:,k]!=0)[0]
        if nonzero_indices.shape[0] !=0:
#             exec('output{}=tf.sigmoid(-1*outputLayer_w{}*tf.expand_dims(r_{}[:,nonzero_indices[0]],1))'
#                  .format(k,j,n_iteration-1))
#             this is false. should be first added,then sigmoid()
            exec('output{}=outputLayer_w{}*tf.expand_dims(r_{}[:,nonzero_indices[0]],1)'
                  .format(k,j,n_iteration-1))

            j+=1
            if nonzero_indices.shape[0] !=1:
                for l in range(1,nonzero_indices.shape[0]):
#                     exec('output{}=output{}+tf.sigmoid(-1*outputLayer_w{}*tf.expand_dims(r_{}[:,nonzero_indices[{}]],1))'
#                          .format(k,k,j,n_iteration-1,l))
                    exec('output{}=output{}+outputLayer_w{}*tf.expand_dims(r_{}[:,nonzero_indices[{}]],1)'
                         .format(k,k,j,n_iteration-1,l))
                    j+=1
            exec('output{}=tf.sigmoid(-1*output{})'.format(k,k))
    exec('y=tf.concat([output{},output{}],1)'.format(0,1))
    for k in range(2,C_lastLayer.shape[1]):
        exec('y=tf.concat([y,output{}],1)'.format(k))
    print(y)


    
            

Tensor("output/concat_62:0", shape=(?, 64), dtype=float32)


In [6]:
len(tf.trainable_variables())

71360

In [6]:
#check paper Nachmani - lerning to decode linear codes using deep learning
learning_rate=tf.placeholder(tf.float32,shape=[])
loss_value=tf.reduce_mean(tf.losses.mean_squared_error(labels=y_,predictions=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_value)
accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.round(y),tf.round(y_)),tf.float32))
ber=tf.reduce_mean(tf.cast(tf.not_equal(y_, tf.round(y)),tf.float32))
BLER=BLER_errors(y_,y)

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.graph.finalize()
#     for i in range(1000):
# #         for j in range(10):
# #             train_step.run(feed_dict={x:x_train[128*j:128*(j+1)],y_:y_train[128*j:128*(j+1)]})
        
#         train_step.run(feed_dict={x:x_train[0:32],y_:y_train[0:32]})
#         print('loss_value_{}: '.format(i),loss_value.eval({x:x_train[0:50,:],y_:y_train[0:50,:]}))
#         print('accuracy_{}: '.format(i),accuracy.eval({x:x_train[0:50,:],y_:y_train[0:50,:]}))
#         print('ber_{}: '.format(i),ber.eval({x:x_train[0:50,:],y_:y_train[0:50,:]}))
#         print('BLER_{}: '.format(i),BLER.eval({x:x_train[0:50,:],y_:y_train[0:50,:]}))
            
        

In [7]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
sess.graph.finalize()
    

In [ ]:
with sess.as_default():
    for i in range(1000):
    #         for j in range(10):
    #             train_step.run(feed_dict={x:x_train[128*j:128*(j+1)],y_:y_train[128*j:128*(j+1)]})
        if i<500:
            train_step.run(feed_dict={x:x_train[0:128,:],y_:y_train[0:128,:],learning_rate:0.7})
            print('loss_value_{}: '.format(i),loss_value.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
            print('accuracy_{}: '.format(i),accuracy.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
            print('ber_{}: '.format(i),ber.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
            print('BLER_{}: '.format(i),BLER.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
        elif i<800:
            train_step.run(feed_dict={x:x_train[0:128,:],y_:y_train[0:128,:],learning_rate:0.5})
            print('loss_value_{}: '.format(i),loss_value.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
            print('accuracy_{}: '.format(i),accuracy.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
            print('ber_{}: '.format(i),ber.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
            print('BLER_{}: '.format(i),BLER.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))   
        else:
            train_step.run(feed_dict={x:x_train[0:128,:],y_:y_train[0:128,:],learning_rate:0.3})
            print('loss_value_{}: '.format(i),loss_value.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
            print('accuracy_{}: '.format(i),accuracy.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
            print('ber_{}: '.format(i),ber.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))
            print('BLER_{}: '.format(i),BLER.eval({x:x_train[0:128,:],y_:y_train[0:128,:]}))

In [16]:
with sess.as_default():
#     output0=output0.eval({x:[x_train[0,:]]})
#     output1=output1.eval({x:[x_train[0,:]]})
    y_predict=y.eval({x:[x_train[0,:]]})

In [17]:
np.round(y_predict)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

# TEST

In [25]:
code_length=input_shape
info_rate=0.5
test_batch = 128  
num_words = 1000      # multiple of test_batch

x_test_temp=np.ones((num_words,code_length),dtype=float)
y_test=np.zeros((num_words,code_length),dtype=float)

# for i in range(num_words):
#     x_test_temp[i,:]=x
#     y_test[i,:]=y


SNR_dB_start_Eb = 0
SNR_dB_stop_Eb = 5
SNR_points = 20
SNR_dB_start_Es = SNR_dB_start_Eb + 10*np.log10(info_rate)
SNR_dB_stop_Es = SNR_dB_stop_Eb + 10*np.log10(info_rate)
sigma_start = np.sqrt(1/(2*10**(SNR_dB_start_Es/10)))
sigma_stop = np.sqrt(1/(2*10**(SNR_dB_stop_Es/10)))
sigmas = np.linspace(sigma_start, sigma_stop, SNR_points)
ber_result=np.zeros(len(sigmas),dtype=float)
temp=0

with sess.as_default():
    for i in range(len(sigmas)):
        x_test=x_test_temp+sigmas[i]*np.random.normal(0,1,(num_words,code_length))
        for j in range((np.floor(num_words/test_batch)).astype(int)):
            temp=temp+ ber.eval({x_test_temp[j*test_batch:(j+1)*test_batch],
                               y_test_temp[j*test_batch:(j+1)*test_batch]})
        ber_result[i]=temp/(j+1)

AttributeError: 'numpy.ndarray' object has no attribute 'eval'

In [5]:
import numpy as np
try1=np.array([1,0,1,1,0,0,0,1],dtype=float)
np.where(try1!=0)[0].shape[0]

4

In [24]:
C_lastLayer.shape[1]

64

In [11]:
try_c=np.array(range(10)).convert_to_tensor()


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [19]:
np.floor(100/3)

33.0

In [24]:
np.floor(num_words/test_batch).astype(int)

7